In [1]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv
from collections import defaultdict
import random
import string
from sklearn import linear_model
from sklearn.linear_model import Ridge
from surprise import SVD
import math
import numpy as np
import dateutil.parser
from collections import Counter
import pandas as pd
import json

### 1. Dataset

We are using the [Amazon Product Reviews dataset](https://cseweb.ucsd.edu/~jmcauley/datasets.html#amazon_reviews) from Prof. McAuley’s datasets. Although the data originally contains 82.83 million unique reviews from 20 million users, we are using the Amazon data from 2014 specifically. 

To minimize the running time of our algorithm, we will work with just the data from a specific category so let's take the "Shoes" data.

#### Reading in the data

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
        if i % 100000 == 0:
            print(i)
    return pd.DataFrame.from_dict(df, orient='index')

In [3]:
shoes_df = getDF("reviews_Clothing_Shoes_and_Jewelry.json.gz")

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000


In [19]:
shoes_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2XVJBSRI3SWDI,0000031887,abigail,"[0, 0]",Perfect red tutu for the price. I baught it as...,5.0,Nice tutu,1383523200,"11 4, 2013"
1,A2G0LNLN79Q6HR,0000031887,"aj_18 ""Aj_18""","[1, 1]",This was a really cute tutu the only problem i...,4.0,Really Cute but rather short.,1337990400,"05 26, 2012"
2,A2R3K1KX09QBYP,0000031887,alert consumer,"[1, 1]",the tutu color was very nice. the only issue w...,2.0,not very good material.,1361059200,"02 17, 2013"
3,A19PBP93OF896,0000031887,"Alinna Satake ""Can't Stop Eating""","[0, 1]",My 3-yr-old daughter received this as a gift f...,1.0,Tiny and Poorly Constructed!,1363824000,"03 21, 2013"
4,A1P0IHU93EF9ZK,0000031887,Amanda,"[0, 0]",Bought it for my daughters first birthday whic...,4.0,i love it,1390435200,"01 23, 2014"


In [17]:
import tqdm
def parse(path):
    g = gzip.open(path, 'rb')
    for l in tqdm.tqdm(g):
        yield eval(l)

def getDFMeta(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [18]:
shoes_df_meta = getDFMeta("meta_Clothing_Shoes_and_Jewelry.json.gz")

1503384it [09:11, 2724.09it/s]


In [20]:
shoes_df_meta.head()

,asin,related,title,price,salesRank,imUrl,brand,categories,description
0,0000037214,"{'also_viewed': ['B00JO8II76', 'B00DGN4R1Q', '...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,6.99,{'Clothing': 1233557},http://ecx.images-amazon.com/images/I/31mCncNu...,Big Dreams,"[Clothing, Shoes & Jewelry, Girls]",NaN
1,0000031887,"{'also_bought': ['0000031852', '0000031895', '...",Ballet Dress-Up Fairy Tutu,6.79,{'Sports &amp; Outdoors': 8547},http://ecx.images-amazon.com/images/I/314qZjYe...,Boutique Cutie,"[Clothing, Shoes & Jewelry, Girls, Clothing, A...",This adorable basic ballerina tutu is perfect ...
2,0123456479,"{'also_bought': ['B000BMTCK6', 'B0006JCGUM', '...",SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,64.98,{'Kitchen & Dining': 16987},http://ecx.images-amazon.com/images/I/413tGhqo...,NaN,"[Clothing, Shoes & Jewelry, Novelty, Costumes ...",Elegance par excellence. Hand-crafted of the f...
3,0456844570,"{'also_viewed': ['B008MTRT1O', 'B00BUG47S4', '...",RiZ Women's Beautify Crafted &frac12; Rimmed F...,NaN,{'Clothing': 1180499},http://ecx.images-amazon.com/images/I/31QZTHxv...,NaN,"[Clothing, Shoes & Jewelry, Women, Accessories...",NaN
4,0456808574,"{'also_viewed': ['B00A6JD8EA', 'B00BFL4FTA', '...",Lantin White Visor Wrap Around Ski Style Aviat...,NaN,{'Clothing': 1038084},http://ecx.images-amazon.com/images/I/31UsrgT5...,NaN,"[Clothing, Shoes & Jewelry, Women, Accessories...",NaN


In [21]:
shoes_df_meta_all_cat = getDFMeta("meta_Clothing_Shoes_and_Jewelry.json.gz")

1503384it [05:55, 4233.51it/s]


### Basic Statistics

#### All data

Review data

In [23]:
len(shoes_df)

5748920

Let's see how many are in each category

#### Shoes data

In [53]:
def cat_counts(df):
    category_counts = defaultdict(int)
    for idx, row in df.iterrows():
        categories = row['categories']
        for cat in categories:
            category_counts[cat] += 1
    return category_counts

In [40]:
def filter_df_by_category(df, category):
    return df[df.apply(lambda x: category in x["categories"], axis=1)]

In [41]:
shoes_df_meta_filter = filter_df_by_category(shoes_df_meta, 'Shoes')

We're working with less data for now

In [43]:
len(shoes_df_meta_filter)

58289

In [44]:
len(shoes_df_meta)

1503384

So let's also filter our review data to those we have metadata about

In [48]:
shoes_asin_set = set(shoes_df_meta_filter['asin'])
len(shoes_asin_set)

58289

In [49]:
def filter_df_by_asin(df):
    return df[df.apply(lambda x: x["asin"] in shoes_asin_set, axis=1)]

In [50]:
shoes_df_filter = filter_df_by_asin(shoes_df)

In [51]:
len(shoes_df_filter)

167084

In [52]:
len(shoes_df)

5748920

In [56]:
shoes_df_meta_filter['categories'] = shoes_df_meta_filter['categories'].apply(lambda x: [cat for cat in x if cat != 'Clothing, Shoes & Jewelry'])

<ipython-input-56-6c78dcff6180>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shoes_df_meta_filter['categories'] = shoes_df_meta_filter['categories'].apply(lambda x: [cat for cat in x if cat != 'Clothing, Shoes & Jewelry'])


In [57]:
category_counts = cat_counts(shoes_df_meta_filter)

creating a dictionary of sets from the "also_bought" feature in the metadata

In [75]:
def also_bought_dict_def(df):
    also_bought_dict = defaultdict(set)
    for idx, row in df.iterrows():
        related = row['related']
        asin = row['asin']
        if type(related) is dict and 'also_bought' in related:
            for asin2 in related['also_bought']:
                also_bought_dict[asin].add(asin2)
    return also_bought_dict

In [78]:
also_bought_dict = also_bought_dict_def(shoes_df_meta_filter)

In [110]:
positives = set()

for key in also_bought_dict:
    bought_together = list(also_bought_dict[key])
    for i in range(len(bought_together)):
        for j in range(i + 1, len(bought_together)):
            if bought_together[i] in shoes_asin_set and bought_together[j] in shoes_asin_set:
                positives.add((bought_together[i], bought_together[j]))

In [114]:
negatives = set()

for i in range(len(positives)):
    sample1 = random.choice(list(shoes_asin_set))
    while(1):
        sample2 = random.choice(list(shoes_asin_set))
        if sample2 in also_bought_dict[sample1]:
            continue
        else:
            negatives.add((sample1, sample2))
            break

In [125]:
items = list(positives) + list(negatives)

In [128]:
pos_1 = [1] * len(positives)
neg_0 = [0] * len(negatives)

In [130]:
items_y = pos_1 + neg_0

In [131]:
X_tr, X_te, y_tr, y_te = train_test_split(items, items_y, test_size=0.2, random_state=0)

In [152]:
train_items = set()

for idx, item in enumerate(X_tr):
    train_items.add(item[0])
    train_items.add(item[1])

In [154]:
itemsPerUser = defaultdict(set)
usersPerItem = defaultdict(set)

In [155]:
for idx, row in shoes_df_filter.iterrows():
    user = row['reviewerID']
    item = row['asin']
    
    if item in train_items:
        usersPerItem[item].add(user)
        itemsPerUser[user].add(item)

In [157]:
jaccard_list = []

for item1, item2 in X_te:    
    jaccard = Jaccard(usersPerItem[item1], usersPerItem[item2])
    jaccard_list.append(jaccard)

In [158]:
predictions = []

for j in jaccard_list:
    if j > 0.01:
        predictions.append(1)
    else:
        predictions.append(0)

In [159]:
accuracy = 0
for a,b in zip(predictions, y_te):
    if a == b:
        accuracy += 1
        
accuracy / len(predictions)

0.5075575027382256

"also bought" but James' way

In [166]:
positives = set()

for idx, b in tqdm.tqdm(shoes_df_meta_filter.iterrows()):
    a1= b["asin"]
    related = b["related"]
    if type(related) is dict and 'also_bought' in related:
        for a2 in related['also_bought']:
            if a2 in shoes_asin_set:
                if a1 < a2:
                    positives.add((a1,a2))
                else:
                    positives.add((a2,a1))
    if type(related) is dict and 'bought_together' in related:
        for a2 in related['bought_together']:
            if a2 in shoes_asin_set:
                if a1 < a2:
                    positives.add((a1,a2))
                if a2 < a1:
                    positives.add((a2,a1))

58289it [00:10, 5472.28it/s]


In [172]:
ones = [1] * len(positives)

In [169]:
negatives = set()

for idx, b in tqdm.tqdm(positives):
    sample1 = random.choice(list(shoes_asin_set))
    while(1):
        sample2 = random.choice(list(shoes_asin_set))
        if (sample1, sample2) in positives or (sample2, sample1) in positives:
            continue
        else:
            negatives.add((sample1, sample2))
            break

100%|██████████| 120058/120058 [12:22<00:00, 161.79it/s]


In [173]:
zeros = [0] * len(negatives)

In [170]:
for idx, row in shoes_df_filter.iterrows():
    user = row['reviewerID']
    item = row['asin']
    
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)

In [174]:
X = list(positives) + list(negatives)

In [176]:
y = ones + zeros

In [177]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [178]:
jaccard_list = []

for item1, item2 in X:    
    jaccard = Jaccard(usersPerItem[item1], usersPerItem[item2])
    jaccard_list.append(jaccard)

In [190]:
predictions = []

for j in jaccard_list:
    if j > 0.5:
        predictions.append(1)
    else:
        predictions.append(0)

In [191]:
accuracy = 0
for a,b in zip(predictions, y):
    if a == b:
        accuracy += 1
        
accuracy / len(predictions)

0.5000041646884397

tldr jaccard or any similarity metric sucks because we do not have enough user data

Latent Factor Model

- [x] Average of the reviews for that item (if any)
- [x] One hot encode the words from the titles
- [x] One hot encode the categories
- Price

In [194]:
from fastFM import als

Calculate average of reviews for every item

In [196]:
rating_per_item = defaultdict(set)

for idx, row in tqdm.tqdm(shoes_df_filter.iterrows()):
    item = row['asin']
    rating = row['overall']
    
    rating_per_item[item].add(rating)

167084it [00:12, 13204.85it/s]


In [198]:
avg_rating_per_item = defaultdict(float)

for key in rating_per_item:
    avg_rating_per_item[key] = sum(rating_per_item[key])/len(rating_per_item[key])

One hot encode categories

In [202]:
category_counts = cat_counts(shoes_df_meta_filter)

In [203]:
len(category_counts)

82

In [205]:
categories_set = set(category_counts)

In [206]:
categories_set_ind = dict()

count = 0
for cat in categories_set:
    categories_set_ind[cat] = count
    count += 1

In [263]:
len(categories_set_ind)

82

One hot encode most popular words from the titles

In [211]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for idx, row in tqdm.tqdm(shoes_df_meta_filter.iterrows()):
    if type(row['title']) is str:
        r = ''.join([c for c in row['title'].lower() if not c in punctuation])
        for w in r.split():
            wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

58289it [00:04, 11932.03it/s]


In [213]:
len(counts)

30300

In [212]:
counts[:10]

[(26642, 'womens'),
 (11491, 'shoes'),
 (8364, 'mens'),
 (7101, 'black'),
 (6406, 'sandals'),
 (6165, 'toe'),
 (5921, 'boots'),
 (5747, 'heel'),
 (5119, 'high'),
 (5044, 'shoe')]

In [257]:
final_counts = counts[:3960]

In [258]:
len(final_counts)

3960

In [259]:
title_words_ind = dict()

count = 0
for num, word in tqdm.tqdm(final_counts):
    title_words_ind[word] = count
    count += 1

100%|██████████| 3960/3960 [00:00<00:00, 877274.80it/s]


In [262]:
len(title_words_ind)

3960

One hot encode the item ids

In [273]:
len(shoes_asin_set)

58289

In [276]:
shoes_asin_ind = dict()

count = 0
for asin in tqdm.tqdm(shoes_asin_set):
    shoes_asin_ind[asin] = count
    count += 1

100%|██████████| 58289/58289 [00:00<00:00, 85103.27it/s]


In [277]:
len(shoes_asin_ind)

58289

In [310]:
(len(title_words_ind) + len(categories_set_ind) + 1) * 2 + len(shoes_asin_ind)

66375

In [312]:
import scipy

# tuple is (number of item pairs we have, column representation)
# (title one hot + categories one hot + avg rating (1)) * 2 because two items + item ids one hot
Xmat = scipy.sparse.lil_matrix((len(X), (len(title_words_ind) + len(categories_set_ind) + 1) * 2 + len(shoes_asin_ind)))
                              )

In [272]:
import random

combine = list(zip(X, y))
random.shuffle(combine)
X_shuff, y_shuff = zip(*combine)

Map items to their row in the metadata dataframe

In [284]:
shoes_df_meta_filter.reset_index(drop=True, inplace=True)

In [287]:
shoes_df_meta_filter.head()

,asin,related,title,price,salesRank,imUrl,brand,categories,description
0,8921463216,"{'also_viewed': ['8921463267', '8921463208', '...",Hello Kitty LALA Lovely Womens Summer Slippers...,NaN,{'Beauty': 377777},http://ecx.images-amazon.com/images/I/41hbUxgB...,NaN,"[Women, Shoes, Slippers]",NaN
1,8921463267,"{'also_viewed': ['8921463259', 'B007682TD0', '...",Hello Kitty LALA Lovely Womens Summer Slippers...,22.99,{'Beauty': 171597},http://ecx.images-amazon.com/images/I/41H1HmaH...,Hello Kitty,"[Women, Shoes, Slippers]",NaN
2,8921463240,"{'also_viewed': ['8921463259', '8921463267', '...",Hello Kitty LALA Lovely Womens Summer Slippers...,22.99,{'Beauty': 763080},http://ecx.images-amazon.com/images/I/41H1HmaH...,Hello Kitty,"[Women, Shoes, Slippers]",NaN
3,B00004U3SF,"{'also_bought': ['B004302N2Q', 'B0017QA998', '...",Wizard of Oz Child's Deluxe Dorothy Ruby Red S...,20.09,{'Toys & Games': 101939},http://ecx.images-amazon.com/images/I/41MWXUHW...,Rubie&#39;s Costume Co,"[Girls, Shoes, Flats]",For any girl who's enchanted by theWizard of O...
4,B00004VWJW,"{'also_viewed': ['B0012YIE3S', 'B000XDRJFI', '...",Birkenstock 68051 Green Super Birki Clog Size ...,71.90,"{'Patio, Lawn & Garden': 253160}",http://ecx.images-amazon.com/images/I/41CHQ5CF...,Flagline,"[Men, Shoes, Mules & Clogs]",There's a delicious satisfaction in slipping o...


In [285]:
item_to_df_index = defaultdict(int)

for idx, b in tqdm.tqdm(shoes_df_meta_filter.iterrows()):
    item_to_df_index[b['asin']] = idx

58289it [00:04, 13719.43it/s]


In [315]:
for i in tqdm.tqdm(range(len(X_shuff))):
    item1, item2 = X_shuff[i]
    
    item1row = shoes_df_meta_filter.iloc[[item_to_df_index[item1]]]
    item2row = shoes_df_meta_filter.iloc[[item_to_df_index[item2]]]
    
    # ITEM 1
    # Avg rating
    avg_rating_1 = [avg_rating_per_item[item1]]
    
    # 1 hot categories
    categories_1 = [0] * len(categories_set_ind)
    cat1 = item1row['categories'].values.tolist()[0]
    for cat in cat1:
        if cat in categories_set_ind:
            categories_1[categories_set_ind[cat]] = 1
    
    # 1 hot title
    title_1 = [0] * len(title_words_ind)
    for word in item1row['title'].values.tostring().split():
        if word in title_words_ind:
            title_1[title_words_ind[word]] = 1
    
    # ITEM 2 
    # Avg rating
    avg_rating_2 = [avg_rating_per_item[item2]]
    
    # 1 hot categories
    categories_2 = [0] * len(categories_set_ind)
    cat2 = item2row['categories'].values.tolist()[0]
    for cat in cat2:
        if cat in categories_set_ind:
            categories_2[categories_set_ind[cat]] = 1
    
    # 1 hot title
    title_2 = [0] * len(title_words_ind)
    for word in item2row['title'].values.tostring().split():
        if word in title_words_ind:
            title_2[title_words_ind[word]] = 1
    
    # one hot encode item ids
    item_ids_enc = [0] * len(shoes_asin_ind)
    item_ids_enc[shoes_asin_ind[item1]] = 1
    item_ids_enc[shoes_asin_ind[item2]] = 1
    
    Xmat[i] = np.asarray(avg_rating_1 + categories_1 + title_1 + avg_rating_2 + categories_2 + title_2 + item_ids_enc)

  0%|          | 0/240114 [00:00<?, ?it/s]<ipython-input-315-3ec89a92058c>:20: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  for word in item1row['title'].values.tostring().split():
<ipython-input-315-3ec89a92058c>:37: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  for word in item2row['title'].values.tostring().split():
100%|██████████| 240114/240114 [37:04<00:00, 107.94it/s]


In [316]:
fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=5, l2_reg_w=0.1, l2_reg_V=0.5)

In [317]:
len(X_shuff) * 0.8

192091.2

In [356]:
X_train,y_train = Xmat[:192091],np.array(y_shuff[:192091])
X_test,y_test = Xmat[192091:],np.array(y_shuff[192091:])

In [357]:
fm.fit(X_train, y_train)

FMRegression(l2_reg_V=0.5, n_iter=1000, rank=5)

In [358]:
y_pred = fm.predict(X_test)

In [359]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [362]:
y_pred_final = []
for x in y_pred:
    if x > 0.5:
        y_pred_final.append(1)
    else:
        y_pred_final.append(0)

In [364]:
MSE(y_pred_final, y_test)

0.05259979593111634

In [365]:
accuracy = 0
for a,b in zip(y_pred_final, y_test):
    if a == b:
        accuracy += 1
        
accuracy / len(y_pred_final)

0.9474002040688837

random ideas

- weighted
    - cosine similarity (sets?)
    - jaccard similarity
- logistic regression
    - for both items
        - one hot encode words from the titles
        - one hot encode the category
        - price
        - description
    - need to replace the nan values!! 
    - jaccard similarity between the two items in the row (this will use the user data)
        - can't really do pearson or cosine
- other (but won't work bc negative validation set)
    - ratings
    - user who did the review
    - top words from the review itself
    - temporal
    
Create training and test sets
- split the items into 80, 20
- create a test set that had another 20% size worth of pairs that were not bought together 

### 2. Predictive Task

Based on our exploratory analysis, the predictive task we are choosing to focus on is given two items i, j, predict whether or not any user has co-purchased them before.